# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import warnings
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql("SELECT * FROM Messages", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # normalization word tokens and remove stop words
    normlizer = PorterStemmer()
    stop_words = stopwords.words("english")
    
    normlized = [normlizer.stem(word) for word in tokens if word not in stop_words]
    
    return normlized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline1 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size = 0.2, random_state = 45)

pipeline1.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def plot_scores(Y_test, Y_pred):
    #Testing the model
    # Printing the classification report for each label
    i = 0
    for col in Y_test:
        print('Feature {}: {}'.format(i+1, col))
        print(classification_report(Y_test[col], Y_pred[:, i]))
        i = i + 1
    accuracy = (Y_pred == Y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

In [7]:
# Prediction: the Random Forest Classifier  
Y_pred = pipeline1.predict(X_test)
plot_scores(Y_test, Y_pred)

Feature 1: related
             precision    recall  f1-score   support

          0       0.62      0.47      0.54      1198
          1       0.85      0.91      0.88      4002
          2       0.36      0.36      0.36        44

avg / total       0.79      0.81      0.80      5244

Feature 2: request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4335
          1       0.80      0.45      0.57       909

avg / total       0.88      0.88      0.87      5244

Feature 3: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5214
          1       0.00      0.00      0.00        30

avg / total       0.99      0.99      0.99      5244

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.75      0.86      0.80      3044
          1       0.75      0.60      0.67      2200

avg / total       0.75      0.75      0.74      5244

Feature 5

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
# Show parameters for the pipline
pipeline1.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f96e04aa620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [9]:
# Using grid search
# Create Grid search parameters for Random Forest Classifier   
parameters = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 20]
}

cv = GridSearchCV(pipeline1, param_grid = parameters)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [10]:
cv


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Fit the first tuned model
cv.fit(X_train, Y_train)

In [13]:
# Predicting using the first tuned model 
Y_pred = cv.predict(X_test)
plot_scores(Y_test, Y_pred)

Feature 1: related
             precision    recall  f1-score   support

          0       0.66      0.44      0.53      1198
          1       0.85      0.93      0.89      4002
          2       0.41      0.45      0.43        44

avg / total       0.80      0.81      0.80      5244

Feature 2: request
             precision    recall  f1-score   support

          0       0.90      0.98      0.94      4335
          1       0.83      0.47      0.60       909

avg / total       0.89      0.89      0.88      5244

Feature 3: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5214
          1       0.00      0.00      0.00        30

avg / total       0.99      0.99      0.99      5244

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.76      0.85      0.81      3044
          1       0.76      0.63      0.69      2200

avg / total       0.76      0.76      0.76      5244

Feature 5

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# using Adaboost Classifier instead 
from sklearn.ensemble import AdaBoostClassifier
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
])

parameters2 =  {'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 60, 70]} 
cv2 = GridSearchCV(pipeline2, param_grid = parameters2)



In [ ]:
cv2

In [ ]:
cv2.fit(X_train, Y_train)

In [38]:
# Predicting using the first tuned model 
Y_pred = cv2.predict(X_test)
plot_scores(Y_test, Y_pred)

Feature 1: related
             precision    recall  f1-score   support

          0       0.65      0.14      0.22      1198
          1       0.79      0.98      0.87      4002
          2       0.45      0.11      0.18        44

avg / total       0.75      0.78      0.72      5244

Feature 2: request
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      4335
          1       0.77      0.53      0.63       909

avg / total       0.88      0.89      0.88      5244

Feature 3: offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      5214
          1       0.00      0.00      0.00        30

avg / total       0.99      0.99      0.99      5244

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.75      0.88      0.81      3044
          1       0.78      0.61      0.68      2200

avg / total       0.76      0.76      0.76      5244

Feature 5

### 9. Export your model as a pickle file

In [39]:
# Create a pickle file for the model
with open('model.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.